#Pre

In [1]:
!pip install osmnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.2/107.2 kB 2.1 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
import os
import re
import json
import osmnx as ox
from itertools import chain

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#New code

###Crawl street

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Năm 2 kì 2/DS108/Lab6/full_detail.csv')
df = pd.DataFrame(columns=["Đường", "Xã/Phường", "Quận/Huyện","Tỉnh/Thành Phố"])

In [ ]:
len(df)

18454

In [ ]:
ind = 1
place_name = data['Phường Xã'][ind] + ', '+ data['Quận Huyện'][ind] + ', ' + data['Tỉnh Thành Phố'][ind] + ', Việt Nam'
place_name

'Phường Trúc Bạch, Quận Ba Đình, Thành phố Hà Nội, Việt Nam'

In [ ]:
def get_streets(place_name):
    graph = ox.graph_from_place(place_name, network_type='all')

    # Get a GeoDataFrame of the edges (streets)
    edges = ox.graph_to_gdfs(graph, nodes=False)

    # Extract and flatten street names
    street_names_raw = edges['name'].dropna()
    street_names_list = list(chain.from_iterable(street_names_raw.apply(lambda x: x if isinstance(x, list) else [x])))

    # Get unique street names
    street_names = pd.Series(street_names_list).unique()
    return street_names

In [ ]:
count = 0
stopper = 100
running = 0
QH_name = ""
for ind in range(0, 4240):
    if(data['Phường Xã'][ind] == ""):
        place_name = data['Quận Huyện'][ind] + ', ' + data['Tỉnh Thành Phố'][ind] + ', Việt Nam'
    else:
        place_name = data['Phường Xã'][ind] + ', '+ data['Quận Huyện'][ind] + ', ' + data['Tỉnh Thành Phố'][ind] + ', Việt Nam'
    if(QH_name != data['Quận Huyện'][ind]):
        QH_name = data['Quận Huyện'][ind]
        QH_flag = 0
    # Download street network data for the defined place
    try:
        street_names = get_streets(place_name)
        QH_flag = 1
        for street_name in street_names:
            df.loc[len(df.index)] = [street_name, data['Phường Xã'][ind], data['Quận Huyện'][ind], data['Tỉnh Thành Phố'][ind]]
    except:
        print(f"{place_name} not found!")
        try:
            if(ind != len(data.index)-1):
                if(QH_flag == 0 and QH_name != data['Quận Huyện'][ind+1]):
                    place_name = data['Quận Huyện'][ind] + ', ' + data['Tỉnh Thành Phố'][ind] + ', Việt Nam'
                    street_names = get_streets(place_name)
                    for street_name in street_names:
                        df.loc[len(df.index)] = [street_name, " ", data['Quận Huyện'][ind], data['Tỉnh Thành Phố'][ind]]
                    print(f"{place_name} crawled")
        except:
            print("we fucked")
    if(ind %106 == 0 and ind != 0):
        running +=1
        count+=1
        print(f"Done {count}%, recorded {len(df)} streets in total")
        df.to_csv('/content/drive/MyDrive/Năm 2 kì 2/DS108/Lab6/new_detail.csv', index=False)
        if(running == stopper):
            print(f"Stopped by stopper at ind = {ind}")
            break

df.to_csv('/content/drive/MyDrive/Năm 2 kì 2/DS108/Lab6/new_detail.csv', index=False)

Done 1%, recorded 6597 streets in total
Xã Nam Sơn, Huyện Sóc Sơn, Thành phố Hà Nội, Việt Nam not found!
Xã Tân Hưng, Huyện Sóc Sơn, Thành phố Hà Nội, Việt Nam not found!
Xã Việt Long, Huyện Sóc Sơn, Thành phố Hà Nội, Việt Nam not found!
Xã Xuân Giang, Huyện Sóc Sơn, Thành phố Hà Nội, Việt Nam not found!
Xã Đức Hoà, Huyện Sóc Sơn, Thành phố Hà Nội, Việt Nam not found!
Xã Bắc Hồng, Huyện Đông Anh, Thành phố Hà Nội, Việt Nam not found!
Xã Nam Hồng, Huyện Đông Anh, Thành phố Hà Nội, Việt Nam not found!
Xã Vân Hà, Huyện Đông Anh, Thành phố Hà Nội, Việt Nam not found!
Xã Vân Nội, Huyện Đông Anh, Thành phố Hà Nội, Việt Nam not found!
Xã Kim Nỗ, Huyện Đông Anh, Thành phố Hà Nội, Việt Nam not found!
Xã Kim Chung, Huyện Đông Anh, Thành phố Hà Nội, Việt Nam not found!
Xã Dục Tú, Huyện Đông Anh, Thành phố Hà Nội, Việt Nam not found!
Xã Đại Mạch, Huyện Đông Anh, Thành phố Hà Nội, Việt Nam not found!
Xã Vĩnh Ngọc, Huyện Đông Anh, Thành phố Hà Nội, Việt Nam not found!
Xã Cổ Loa, Huyện Đông Anh, Thàn

##Adress parsing

In [4]:
raw_street_data = pd.read_csv('/content/drive/MyDrive/Năm 2 kì 2/DS108/Lab6/full_detail.csv')
street_data = raw_street_data.sort_values(by=raw_street_data.columns[0])
street_data

,Đường,Xã/Phường,Quận/Huyện,Tỉnh/Thành Phố
71811,"""Đường Mươi Phèo -Tư Cò""",Xã Phước Lợi,Huyện Bến Lức,Tỉnh Long An
71549,',Xã Tân Phú,Huyện Đức Hòa,Tỉnh Long An
71587,',Xã Hòa Khánh Nam,Huyện Đức Hòa,Tỉnh Long An
17699,01 Nguyễn Văn Trỗi,Phường Ngọc Trạo,Thành phố Thanh Hóa,Tỉnh Thanh Hóa
32713,01 Đồi Trà,Phường 8,Thành phố Đà Lạt,Tỉnh Lâm Đồng
...,...,...,...,...
17443,Ỷ Lan,Phường Đông Thọ,Thành phố Thanh Hóa,Tỉnh Thanh Hóa
28468,Ỷ Lan,Phường Trần Hưng Đạo,Thành phố Quy Nhơn,Tỉnh Bình Định
32028,Ỷ Lan,,Thành phố Pleiku,Tỉnh Gia Lai
28428,Ỷ Lan,Phường Thị Nại,Thành phố Quy Nhơn,Tỉnh Bình Định


In [5]:
data_need_clean = pd.read_excel('/content/drive/MyDrive/Năm 2 kì 2/DS108/Lab6/address_full_0712.xlsx', index_col = 0)
data_need_clean = data_need_clean.iloc[101:, :]
data_clean = data_need_clean.reset_index(drop=True)
data_clean

,Address,tinh,tinh_cat,qh,qh_cat,px,px_cat,duong,Address_ch
0,"chung cư k26 đường dương quảng hàm , phường 7 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chung cư k26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"địa chỉ : 50 đường 42 , phường bình trưng tây ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89 thịnh liệt hoàng mai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"mt 23 trương định , q3",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5423,green town bình tân block a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5424,"chung cư tòa 34t hoàang đạo thúy , khu đô thị ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5425,"27 / 1a , lê thị hồng gấm , p6 , tp . mỹ tho ,...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5426,"tọa lạc 33 nguyễn hữu thọ , phường tân hưng , ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Clean unnecessary infor

In [6]:
custom_punctuation = '!"#$%&\'()*+-,.:;<=>?@[\\]^_`{|}~”“ '

In [7]:
def replace_duong_so(substring):
    # Use regular expression to find "đường xx" and replace with "đường số xx"
    return re.sub(r'đường (\d+)', r'đường số \1', substring)
def add_space(value):
    return value + ' '
for x in range(len(data_clean)):
    data_clean['Address'][x] = data_clean['Address'][x].replace('Địa chỉ',"").replace('địa chỉ',"").replace('tọa lạc',"").replace('Tọa lạc',"").replace(",","").replace(".","").strip(custom_punctuation)
    data_clean['Address'][x] = replace_duong_so(data_clean['Address'][x])
data_clean['Address'] = data_clean['Address'].apply(add_space)
data_clean

,Address,tinh,tinh_cat,qh,qh_cat,px,px_cat,duong,Address_ch
0,chung cư k26 đường dương quảng hàm phường 7 ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,chung cư k26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,50 đường số 42 phường bình trưng tây quận 2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,89 thịnh liệt hoàng mai,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mt 23 trương định q3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
5423,green town bình tân block a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5424,chung cư tòa 34t hoàang đạo thúy khu đô thị t...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5425,27 / 1a lê thị hồng gấm p6 tp mỹ tho tiền...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5426,33 nguyễn hữu thọ phường tân hưng quận 7 tp...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


###Make dict


In [8]:
data = pd.read_excel('/content/drive/MyDrive/Năm 2 kì 2/DS108/Lab6/Danh sách cấp tỉnh kèm theo quận huyện, phường xã ___16_05_2024.xls')
data.drop(columns = ['Tên Tiếng Anh', 'Cấp'], inplace = True)
data['Phường Xã'] = data['Phường Xã'].fillna('')
data['Mã PX'] = data['Mã PX'].fillna(0)
data['Mã PX'] = data['Mã PX'].astype(np.int64)
data

WARNING *** file size (2453381) not 512 + multiple of sector size (512)


,Tỉnh Thành Phố,Mã TP,Quận Huyện,Mã QH,Phường Xã,Mã PX
0,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Phúc Xá,1
1,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Trúc Bạch,4
2,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Vĩnh Phúc,6
3,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Cống Vị,7
4,Thành phố Hà Nội,1,Quận Ba Đình,1,Phường Liễu Giai,8
...,...,...,...,...,...,...
10595,Tỉnh Cà Mau,96,Huyện Ngọc Hiển,973,Xã Viên An Đông,32239
10596,Tỉnh Cà Mau,96,Huyện Ngọc Hiển,973,Xã Viên An,32242
10597,Tỉnh Cà Mau,96,Huyện Ngọc Hiển,973,Thị trấn Rạch Gốc,32244
10598,Tỉnh Cà Mau,96,Huyện Ngọc Hiển,973,Xã Tân Ân,32245


In [13]:
# Tạo một dict mapping giữa tên tỉnh thành phố -> mã tỉnh thành phố && quận huyện -> mã quận huyện && phường xã -> mã phường xã
def mapping_tinhtp_ma(df):
    tinhtp = df['Tỉnh Thành Phố'].unique()
    for i in range(len(tinhtp)):
        tinhtp[i] = tinhtp[i].lower()
    ma_tinhtp = df['Mã TP'].unique()
    return dict(zip(tinhtp, ma_tinhtp))


def mapping_qh_ma(df):
    qh = df['Quận Huyện'].unique()
    for i in range(len(qh)):
        qh[i] = qh[i].lower()
    ma_qh = df['Mã QH'].unique()
    return dict(zip(qh, ma_qh))


def mapping_px_ma(df):
    px = df['Phường Xã'].unique()
    for i in range(len(px)):
        px[i] = px[i].lower()
    ma_px = df['Mã PX'].unique()
    return dict(zip(px, ma_px))


dict_tinhtp_ma = mapping_tinhtp_ma(data)
dict_qh_ma = mapping_qh_ma(data)
dict_px_ma = mapping_px_ma(data)

In [10]:
dict_qh_ma

{1: 'quận ba đình',
 2: 'quận hoàn kiếm',
 3: 'quận tây hồ',
 4: 'quận long biên',
 5: 'quận cầu giấy',
 6: 'quận đống đa',
 7: 'quận hai bà trưng',
 8: 'quận hoàng mai',
 9: 'quận thanh xuân',
 16: 'huyện sóc sơn',
 17: 'huyện đông anh',
 18: 'huyện gia lâm',
 19: 'quận nam từ liêm',
 20: 'huyện thanh trì',
 21: 'quận bắc từ liêm',
 250: 'huyện mê linh',
 268: 'quận hà đông',
 269: 'thị xã sơn tây',
 271: 'huyện ba vì',
 272: 'huyện phúc thọ',
 273: 'huyện đan phượng',
 274: 'huyện hoài đức',
 275: 'huyện quốc oai',
 276: 'huyện thạch thất',
 277: 'huyện chương mỹ',
 278: 'huyện thanh oai',
 279: 'huyện thường tín',
 280: 'huyện phú xuyên',
 281: 'huyện ứng hòa',
 282: 'huyện mỹ đức',
 24: 'thành phố hà giang',
 26: 'huyện đồng văn',
 27: 'huyện mèo vạc',
 28: 'huyện yên minh',
 29: 'huyện quản bạ',
 30: 'huyện vị xuyên',
 31: 'huyện bắc mê',
 32: 'huyện hoàng su phì',
 33: 'huyện xín mần',
 34: 'huyện bắc quang',
 35: 'huyện quang bình',
 40: 'thành phố cao bằng',
 42: 'huyện bảo lâm

###Other

In [14]:
tinh_thanhpho = ['hn','hcm']
dict_number_lama = {'i': "1", 'ii': '2', 'iii': '3', 'iv': '4', 'v': '5', 'vi': '6', 'vii': '7', 'viii': '8', 'ix': '9', 'x': '10'}
for x in dict_tinhtp_ma.keys():
    tinh_thanhpho.append(x.replace("thành phố ","").replace("tỉnh ",""))
quan_huyen = ['2', '9']
for x in dict_qh_ma.keys():
    quan_huyen.append(x.replace("huyện ","").replace("thành phố ","").replace("quận ","").replace("thị xã ",""))
phuong_xa = []
for x in dict_px_ma.keys():
    phuong_xa.append(x.replace("thị trấn ","").replace("phường ","").replace("xã ",""))
phuong_xa = [x for x in phuong_xa if x != ""]
for i in range(len(phuong_xa)):
    l_number_lama = list(dict_number_lama.keys())
    for number_lama in l_number_lama:
        if re.search(number_lama, phuong_xa[i]) != None:
            start = re.search(number_lama, phuong_xa[i]).start()
            end = re.search(number_lama, phuong_xa[i]).end()
            if end == len(phuong_xa[i]):
                phuong_xa[i] = phuong_xa[i].replace(number_lama, dict_number_lama[number_lama])
                break
phuong_xa = [x for x in phuong_xa if x != ""]

In [15]:
for x in range(len(quan_huyen)):
    if(quan_huyen[x] == "2"):
        print("a")

a


In [16]:
#REMOVE PREFIX
def remove_prefix_ttp(text, prefix):
    return text.replace(prefix, "").replace("thành phố","").replace("tp","").replace("tỉnh","")
def remove_prefix(text, prefix):
    return text.replace(prefix,"")
def remove_prefix_qh(text, prefix):
    return text.replace(prefix, "").replace("quận","").replace("q ","").replace("huyện","").replace("thị xã","").replace("tx","").replace("thành phố","").replace("tp","")
def remove_prefix_px(text, prefix):
    return text.replace(prefix, "").replace("phường","").replace("p ","").replace("xã","").replace("thị trấn","").replace("tt","")

#FIND CATEGORY
def find_tinh_cat(prefix):
    if prefix == "hcm" or "hồ chí minh" or "tphcm":
        return 79
    elif prefix == "hn" or "hà nội":
        return 1
    else:
        try:
            ma = dict_tinhtp_ma['tỉnh '+prefix]
        except:
            print(f"tỉnh/thành phố không rõ {prefix}")
            ma = 0
        return ma
def find_qh_cat(prefix):
    try:
        ma = dict_qh_ma['huyện '+prefix]
    except:
        try:
            if ('quận '+prefix=='quận 9') or ('quận '+prefix == 'quận 2'):
                return dict_qh_ma['thành phố thủ đức']
            ma = dict_qh_ma['quận '+prefix]
        except:
            try:
                ma = dict_qh_ma['thị xã '+prefix]
            except:
                try:
                    ma = dict_qh_ma['thành phố '+prefix]
                except:
                    print(f"quận/huyện không rõ {prefix}")
                    ma = 0
    return ma
def find_px_cat(prefix):
    try:
        ma = dict_px_ma['phường '+prefix]
    except:
        try:
            ma = dict_px_ma['xã '+prefix]
        except:
            try:
                ma = dict_px_ma['thị trấn '+prefix]
            except:
                print(f"phường/xã không rõ {prefix}")
                ma = 0
    return ma

#FIND NAME
def find_ttp(x):
    tinh = "None"
    tinh_cat = "0"
    for prefix in tinh_thanhpho:
        x = x.lower()
        loc_prefix = x.find(prefix)
        loc_duong = x.find("đường")
        if(loc_prefix != -1 and loc_prefix - 6 != loc_duong):
            if prefix == "hcm":
                tinh = "hồ chí minh"
            elif prefix == "hn":
                tinh = "hà nội"
            else:
                tinh = prefix
            tinh_cat = find_tinh_cat(prefix)
            x = remove_prefix_ttp(x, prefix)
            break;
    return x, tinh, tinh_cat
def find_qh(x):
    qh = "None"
    qh_cat = "0"
    x = x.lower()
    loc_duong = x.find("đường")
    for prefix in quan_huyen:
        loc_prefix = -1
        flag = False
        try:
            if int(prefix) in range(0,30):
                flag = True
                if (x.find("q" + prefix) !=-1):
                    loc_prefix = x.find("q" + prefix + " ")
                    if(loc_prefix != -1):
                        remove = "q" + prefix
                elif (x.find("q " + prefix) != -1):
                    loc_prefix = x.find("q " + prefix + " ")
                    if(loc_prefix != -1):
                        remove = "q " + prefix
                elif (x.find("quận " + prefix) != -1):
                    loc_prefix = x.find("quận " + prefix + " ")
                    if(loc_prefix != -1):
                        remove = "quận " + prefix
                elif (x.find("quận" + prefix) !=-1):
                    loc_prefix = x.find("quận" + prefix + " ")
                    if(loc_prefix != -1):
                        remove = "quận" + prefix
                if(loc_prefix != -1 and loc_prefix - 6 != loc_duong):
                    qh = "quận "+prefix
                    x = remove_prefix(x, remove)
        except:
            if(flag == False):
                loc_prefix = x.find(" "+prefix+" ")
                if(loc_prefix != -1 and loc_prefix - 6 != loc_duong):
                    qh = prefix
                    x = remove_prefix_qh(x, prefix)
        if(loc_prefix != -1 and loc_prefix - 6 != loc_duong):
            qh_cat = find_qh_cat(prefix)
            break;

    return x, qh, qh_cat
def find_px(x):
    px = "None"
    px_cat = "0"
    x = x.lower()
    loc_duong = x.find("đường")
    for prefix in phuong_xa:
        loc_prefix = -1
        flag = False
        try:
            if int(prefix) in range(0,30):
                flag = True
                if (x.find("p" + prefix) !=-1):
                    loc_prefix = x.find("p" + prefix +" ")
                    if(loc_prefix != -1):
                        remove = "p" + prefix
                elif (x.find("p " + prefix) != -1):
                    loc_prefix = x.find("p " + prefix +" ")
                    if(loc_prefix != -1):
                        remove = "p " + prefix
                elif (x.find("phường " + prefix) != -1):
                    loc_prefix = x.find("phường " + prefix +" ")
                    if(loc_prefix != -1):
                        remove = "phường" + prefix
                elif (x.find("phường" + prefix) !=-1):
                    loc_prefix = x.find("phường" + prefix +" ")
                    if(loc_prefix != -1):
                        remove = "phường " + prefix
                if(loc_prefix != -1 and loc_prefix - 6 != loc_duong):
                    px = "phường "+prefix
                    x = remove_prefix(x, remove)
        except:
            if(flag == False):
                loc_prefix = x.find(" "+prefix+" ")
                if(loc_prefix != -1 and loc_prefix - 6 != loc_duong):
                    px = prefix
                    x = remove_prefix_px(x, prefix)
        if(loc_prefix != -1 and loc_prefix - 6 != loc_duong):
            px_cat = find_px_cat(prefix)
            break;
    return x, px, px_cat

#GENERATE SUBSTRINGS
def generate_substrings(address):
    words = address.split()
    n = len(words)
    substrings = []

    # Generate all substrings with decreasing word count (at least 2 words)
    for length in range(n, 1, -1):
        for start in range(n - length + 1):
            substrings.append(' '.join(words[start:start + length]))

    return substrings

#GET STREET
def get_street(address, tinh, qh, px):
    tinh = "None"
    tinh_cat = 0
    qh = "None"
    qh_cat = 0
    px = "None"
    px_cat = 0
    street = "None"
    result = generate_substrings(address)
    dummy = street_data.loc[street_data['Đường'].str.lower() == 'dummy']
    for substring in result:
        streets_found = street_data.loc[street_data['Đường'].str.lower() == substring]
        if(len(streets_found)!=0):
            dummy = pd.concat([dummy, streets_found])
    dummy = dummy.reset_index(drop=True)

    #format dummy
    if(len(dummy)>0):
        for i in range(len(dummy)):
            dummy['Tỉnh/Thành Phố'][i] = dummy['Tỉnh/Thành Phố'][i].lower().replace("thành phố ","").replace("tỉnh ","")
            dummy['Quận/Huyện'][i] = dummy['Quận/Huyện'][i].lower().replace("huyện ","").replace("thành phố ","").replace("quận ","").replace("thị xã ","")
            try:
                if int(dummy['Quận/Huyện'][i]) in range(0, 30):
                    dummy['Quận/Huyện'][i] = "quận "+ dummy['Quận/Huyện'][i]
            except:
                dummy['Quận/Huyện'][i]
            if (type(dummy['Xã/Phường'][i]) != float):
                dummy['Xã/Phường'][i] = dummy['Xã/Phường'][i].lower().replace("thị trấn ","").replace("phường ","").replace("xã ","")
            try:
                if int(dummy['Xã/Phường'][i]) in range(0, 30):
                    dummy['Xã/Phường'][i] = "phường "+ dummy['Xã/Phường'][i]
            except:
                dummy['Xã/Phường']
    #Get street name list:
    if(tinh != "None"):
        exp_dum = dummy[dummy['Tỉnh/Thành Phố'] == tinh]
        if(len(exp_dum != 0)):
            dummy = exp_dum
    if(qh != "None"):
        exp_dum = dummy[dummy['Quận/Huyện']==qh]
        if(len(exp_dum != 0)):
            dummy = exp_dum
    if(px != "None"):
        exp_dum = dummy[dummy['Xã/Phường']==px]
        if(len(exp_dum != 0)):
            dummy = exp_dum
    dummy = dummy.reset_index(drop=True)

    #Calculate similar score: choose best score
    best = 0
    loc_street = 9999
    loc_tinh = 9999
    loc_qh = 9999
    loc_px = 9999
    score = []
    if(len(dummy)>0):
        for i in range(len(dummy)):
            score.append(0)
            tinh = dummy['Tỉnh/Thành Phố'][i].lower()
            qh = dummy['Quận/Huyện'][i].lower()
            px = dummy['Xã/Phường'][i].lower()
            street = dummy['Đường'][i].lower()
            if (loc_tinh > address.find(tinh) and address.find(tinh)!= -1):
                loc_tinh = address.find(tinh)
            if (loc_qh > address.find(qh) and address.find(qh)!= -1):
                loc_qh = address.find(qh)
            if (loc_px > address.find(px) and address.find(px)!= -1):
                loc_px = address.find(px)
            if (loc_street > address.find(street) and address.find(street)!= -1):
                loc_street = address.find(street)
        for i in range(len(dummy)):
            tinh = dummy['Tỉnh/Thành Phố'][i].lower()
            qh = dummy['Quận/Huyện'][i].lower()
            px = dummy['Xã/Phường'][i].lower()
            street = dummy['Đường'][i].lower()
            if (loc_tinh == address.find(tinh)):
                score[i] += 1
            if (loc_qh == address.find(qh)):
                score[i] += 1
            if (loc_px == address.find(px)):
                score[i] += 1
            if (loc_street == address.find(street)):
                score[i] += 1
        max_value = max(score)
        best = score.index(max_value)
        tinh = dummy['Tỉnh/Thành Phố'][best]
        tinh_cat = find_tinh_cat(tinh.replace("thành phố ","").replace("tỉnh ",""))
        qh = dummy['Quận/Huyện'][best]
        qh_cat = find_qh_cat(qh.replace("huyện ","").replace("thành phố ","").replace("quận ","").replace("thị xã ",""))
        px = dummy['Xã/Phường'][best]
        px_cat = find_px_cat(px.replace("thị trấn ","").replace("phường ","").replace("xã ",""))
        street = dummy['Đường'][best].lower()
    return tinh, tinh_cat, qh, qh_cat, px, px_cat, street

In [17]:
for i in range(len(data_clean)):
    try:
        address = data_clean.loc[i]['Address']
        address, tinh, tinh_cat = find_ttp(address)
        address, qh, qh_cat = find_qh(address)
        address, px, px_cat = find_px(address)
        address = data_clean.loc[i]['Address']
        tinh, tinh_cat, qh, qh_cat, px, px_cat, street = get_street(address, tinh, qh, px)
        data_clean.loc[i, 'tinh'] = tinh
        data_clean.loc[i, 'tinh_cat'] = tinh_cat
        data_clean.loc[i, 'qh'] = qh
        data_clean.loc[i, 'qh_cat'] = qh_cat
        data_clean.loc[i, 'px'] = px
        data_clean.loc[i, 'px_cat'] = px_cat
        data_clean.loc[i, 'duong'] = street
        data_clean.loc[i, 'Address_ch'] = address.replace(tinh,"").replace(qh,"").replace(px,"").strip(custom_punctuation)
    except:
        print("error")

phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
error
error
phường/xã không rõ  
error
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
error
phường/xã không rõ  
phường/xã không rõ  
error
phường/xã không rõ  
error
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
error
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
error
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
error
phường/xã không rõ  
phường/xã không rõ  
error
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
error
phường/xã không rõ  
phường/xã không rõ  
phường/xã không rõ  
phường/xã 

In [19]:
data_clean.to_excel("/content/drive/MyDrive/Năm 2 kì 2/DS108/Lab6/completed_data.xlsx", sheet_name="sheet1", index=False)

In [ ]:
remain_address = []
address = [
    '50 đường số 42 phường bình trưng tây quận 2 '
]
for x in address:
    x, tinh, tinh_cat = find_ttp(x)
    x, qh, qh_cat = find_qh(x)
    x, px, px_cat = find_px(x)
    remain_address.append(x.strip(custom_punctuation))
    print("before get_street")
    print(tinh, tinh_cat, qh, qh_cat, px, px_cat)
    tinh,tinh_cat, qh,qh_cat, px,px_cat, street = get_street(address, tinh, qh, px)
    print(tinh,tinh_cat, qh,qh_cat, px,px_cat, street)
    print("after get_street")

None 0 quận 2 767 bình trưng tây 21865
['50 đường số 42']
